### Используемые библиотеки

In [ ]:
import pywt
from enum import Enum
import cv2
from PIL import Image, ImageChops
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Tuple, Dict, Any

In [ ]:
class ColorSpaceType(Enum):
    RGB = 'RGB'
    YCrCb = 'YCrCb'

class SubbandType(Enum):
    LL = "LL"
    LH = "LH"
    HL = "HL"
    HH = "HH"

def get_image_channels(image_path: str, color_space: ColorSpaceType):
    if color_space is ColorSpaceType.RGB:
        img = Image.open(image_path).convert('RGB')
        img_array = np.array(img)

        return {
            'color_space': color_space,
            'channels': img_array
        }
    
    elif color_space is ColorSpaceType.YCrCb:
        img = Image.open(image_path).convert('RGB')
        img_array = np.array(img)
        img_ycbcr = cv2.cvtColor(img_array, cv2.COLOR_RGB2YCrCb)

        return {
            'color_space': color_space,
            'channels': img_ycbcr
        }
    
    else:
        raise ValueError('Invalid color space.')
        
def save_image_channels(img_data: np.ndarray, out_path: str, color_space: ColorSpaceType):
    if color_space is ColorSpaceType.RGB:
        Image.fromarray(img_data).save(out_path)
        return
    
    elif color_space is ColorSpaceType.YCrCb:
        cbcr_image_data = cv2.cvtColor(img_data, cv2.COLOR_YCrCb2RGB)
        Image.fromarray(cbcr_image_data).save(out_path)
        return
    
    else:
        raise ValueError('Invalid color space.')

## Анализ изображений

### Выделение отличий

In [ ]:
def image_diff(img1: np.ndarray, img2: np.ndarray, amplify: int = 10) -> np.ndarray:
    """
    Создаёт diff двух изображений с усилением различий.

    amplify — множитель, чтобы лучше выделять слабые различия.
    """
    pil1 = Image.fromarray(img1)
    pil2 = Image.fromarray(img2)
    diff = ImageChops.difference(pil1, pil2)

    # Усиливаем различия
    diff = diff.point(lambda p: min(255, p * amplify))

    return np.array(diff)


def image_mask(img1: np.ndarray, img2: np.ndarray) -> np.ndarray:
    """
    Создаёт бинарную маску различий (красные пиксели — есть разница).
    """
    mask = np.any(img1 != img2, axis=-1)

    heatmap = np.zeros_like(img1)
    heatmap[mask] = [255, 0, 0]

    return heatmap


### Вычисление PSNR

In [ ]:
def calculate_psnr(img1: np.ndarray, img2: np.ndarray) -> float:
    """
    Простой PSNR для RGB или grayscale.
    """
    if img1.shape != img2.shape:
        raise ValueError(f"Размеры не совпадают: {img1.shape} != {img2.shape}")

    mse = np.mean((img1.astype(float) - img2.astype(float)) ** 2)
    if mse == 0:
        return float('inf')

    return 10 * np.log10((255.0 ** 2) / mse)


def calculate_psnr_detailed(img1: np.ndarray, img2: np.ndarray) -> Dict[str, Any]:
    """
    PSNR по каналам + средний PSNR.
    """
    if img1.shape != img2.shape:
        raise ValueError("Размеры изображений не совпадают")

    if img1.ndim == 2:
        # 1 канал
        mse = np.mean((img1.astype(float) - img2.astype(float)) ** 2)
        psnr = float('inf') if mse == 0 else 10 * np.log10((255.0 ** 2) / mse)
        return {"psnr": psnr, "mse": mse}

    results = {}
    names = ["R", "G", "B"]

    psnrs = []
    mses = []

    for i, ch in enumerate(names):
        c1 = img1[:, :, i].astype(float)
        c2 = img2[:, :, i].astype(float)

        mse = np.mean((c1 - c2) ** 2)
        psnr = float('inf') if mse == 0 else 10 * np.log10((255.0 ** 2) / mse)

        results[f"psnr_{ch}"] = psnr
        results[f"mse_{ch}"] = mse

        psnrs.append(psnr)
        mses.append(mse)

    results["psnr_avg"] = np.mean(psnrs)
    results["mse_avg"] = np.mean(mses)

    return results

### Полный анализ изображений и визуализация

In [ ]:
def visualize_comparison(
        original: np.ndarray,
        modified: np.ndarray,
        diff: np.ndarray,
        mask: np.ndarray,
        figsize: Tuple[int, int] = (16, 8)
):
    """
    Рисует 4 изображения: исходное, модифицированное, diff и маску различий.
    """
    plt.figure(figsize=figsize)

    plt.subplot(1, 4, 1)
    plt.title("Original")
    plt.imshow(original)
    plt.axis("off")

    plt.subplot(1, 4, 2)
    plt.title("Modified")
    plt.imshow(modified)
    plt.axis("off")

    plt.subplot(1, 4, 3)
    plt.title("Diff (amplified)")
    plt.imshow(diff)
    plt.axis("off")

    plt.subplot(1, 4, 4)
    plt.title("Difference Mask")
    plt.imshow(mask)
    plt.axis("off")

    plt.tight_layout()
    plt.show()


def analyze_images(path1: str, path2: str, amplify: int = 10) -> Dict[str, Any]:
    """
    Полный анализ двух изображений:
    - diff
    - маска различий
    - PSNR
    - PSNR по каналам

    Возвращает словарь с результатами.
    """
    img1 = Image.open(path1).convert("RGB")
    img2 = Image.open(path2).convert("RGB")
    img_data1 = np.array(img1)
    img_data2 = np.array(img2)

    diff = image_diff(img_data1, img_data2, amplify)
    mask = image_mask(img_data1, img_data2)

    psnr = calculate_psnr(img_data1, img_data2)
    psnr_detailed = calculate_psnr_detailed(img_data1, img_data2)

    visualize_comparison(img_data1, img_data2, diff, mask)

    return {
        "psnr": psnr,
        "psnr_detailed": psnr_detailed,
        "diff": diff,
        "mask": mask
    }

## Функции встраивания текста в изображение и получение теста из изображения

In [ ]:
def embed_text(image_path: str, out_path: str, text: str, color_space: ColorSpaceType = ColorSpaceType.RGB, target_subband: str = 'LL', channel: int = 0) -> dict:
    # 1. Получаем выбранный канал изображения
    img_data = get_image_channels(image_path, color_space)
    channel_data = img_data['channels'][:, :, channel].astype(np.float64)

    # 2. Применяем к каналу DWT (используем symmetric для более стабильного поведения)
    wt_type = 'haar'
    wt_mode = 'symmetric'
    
    LL, (LH, HL, HH) = pywt.dwt2(channel_data, wt_type, mode=wt_mode)
    coeffs_dict = {
        'LL': LL,
        'LH': LH,
        'HL': HL,
        'HH': HH
    }

    # 3. Берем выбранную подполосу и приводим ее к одномерному массиву дл удобства
    selected_subband: np.ndarray[np.float64] = coeffs_dict[target_subband]
    subband = selected_subband.flatten()

    # 4. Модифициурем контейнер (одномерный массив)
    #

    # 5. Выполняем обратные преобразования для получения модифицированного канала
    modied_subband = subband.reshape(selected_subband.shape)
    coeffs_dict[target_subband] = modied_subband
    modified_coeffs = (coeffs_dict['LL'], 
                           (coeffs_dict['LH'], 
                            coeffs_dict['HL'], 
                            coeffs_dict['HH']))
    modified_channel_data = pywt.idwt2(modified_coeffs, wt_type, mode=wt_mode)

    # 6. Сохраняем модифицированный канал в изображении
    img_data['channels'][:, :, channel] = modified_channel_data
    save_image_channels(img_data['channels'], out_path, color_space)

    return {
        'image_data': img_data,
        'selected_channel': channel,
        'channel_data': channel_data,
        'target_subband': target_subband
    }

In [ ]:
def extract_text(image_path: str, color_space: ColorSpaceType = ColorSpaceType.RGB, target_subband: str = 'LL', channel: int = 0) -> dict:
    img_data = get_image_channels(image_path, color_space)
    channel_data = img_data['channels'][:, :, channel].astype(np.float64)

    return {
        'image_data': img_data,
        'selected_channel': channel,
        'channel_data': channel_data,
        'target_subband': target_subband,
    }

## Демонстрация работы

### Подготовка данных

In [ ]:
# used_img_path = 'img/marsik-150x200.png'
# embedded_img_path = 'embedded/marsik-150x200-embedded.png'

used_img_path = 'img/marsik-960x1280.png'
embedded_img_path = 'embedded/marsik-960x1280-embedded.png'

secret_string = "Silence in an era of noise: How to find yourself in a world that never stops Our world is immersed in a continuous, intrusive hum. This is not just the physical noise of megacities, but a fundamental information and social backdrop that has become our new habitat. We wake up to the vibration of our smartphones, scroll through our news feeds at breakfast, and are immersed in a whirlwind of notifications, messaging apps, and endless online meetings at work. In the evening, as we try to relax, we unconsciously scroll through social media, consuming the carefully curated lives of others. This permanent digital noise creates the illusion of hyper-connection, but paradoxically leads to deep loneliness and distraction. We know more about the world, but less about ourselves. We have thousands of \"friends,\" but sometimes we don't have anyone to share our true sadness or joy with."

### Встраивание и получения встроенного изображения

In [ ]:
embedded_result = embed_text(
    used_img_path,
    embedded_img_path,
    secret_string,
    color_space=ColorSpaceType.YCrCb,
    target_subband='HL',
    channel=0
)

extract_result = extract_text(
    embedded_img_path,
    color_space=embedded_result['image_data']['color_space'],
    target_subband=embedded_result['target_subband'],
    channel=embedded_result['selected_channel']
)

### Анализ результатов встраивания

In [ ]:
result = analyze_images(
    used_img_path,
    embedded_img_path,
    amplify=5
)

print("PSNR:", result["psnr"])
print(result["psnr_detailed"])